<a href="https://colab.research.google.com/github/Sunnn-y/NaturalLanguageProcessing/blob/main/20231018_%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81_%EC%8B%A4%EC%8A%B5(LDA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDA

## 실습 2
- 영화 리뷰 데이터를 pyLDAvis를 사용해서 토픽모델링을 진행해보고,
다음 데이터 중 부정적인 리뷰를 몇개의 토픽으로 분리하는 것이 가장 적절한지 적절한 토픽 개수를 찾아보세요

In [23]:
import warnings
# 경고 메시지 끄기
warnings.filterwarnings(action='ignore')

# 참고. 다시 출력하게 하기
# warnings.filterwarnings(action='default')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/핀테크 데이터분석/자연어 처리/review.csv',
                 sep = '\t')
# df = df[:100]
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 토큰화 및 데이터 전처리

In [ ]:
!pip install konlpy

In [ ]:
# 토큰화 : Okt 사용
from konlpy.tag import Okt
okt = Okt()
okt_tokens = []

for text in df['document']:
  okt_tokens.append(okt.morphs(text))

okt_tokens

[['아', '더빙', '..', '진짜', '짜증나네요', '목소리'],
 ['흠',
  '...',
  '포스터',
  '보고',
  '초딩',
  '영화',
  '줄',
  '....',
  '오버',
  '연기',
  '조차',
  '가볍지',
  '않구나'],
 ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다'],
 ['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '의',
  '익살스런',
  '연기',
  '가',
  '돋보였던',
  '영화',
  '!',
  '스파이더맨',
  '에서',
  '늙어',
  '보이기만',
  '했던',
  '커스틴',
  '던스트',
  '가',
  '너무나도',
  '이뻐',
  '보였다'],
 ['막',
  '걸음',
  '마',
  '뗀',
  '3',
  '세',
  '부터',
  '초등학교',
  '1',
  '학년',
  '생인',
  '8',
  '살용',
  '영화',
  '.',
  'ㅋㅋㅋ',
  '...',
  '별',
  '반개',
  '도',
  '아까',
  '움',
  '.'],
 ['원작', '의', '긴장감', '을', '제대로', '살려내지못', '했다', '.'],
 ['별',
  '반개',
  '도',
  '아깝다',
  '욕',
  '나온다',
  '이응경',
  '길용우',
  '연',
  '기',
  '생활',
  '이',
  '몇',
  '년',
  '인지',
  '..',
  '정말',
  '발',
  '로',
  '해도',
  '그것',
  '보단',
  '낫겟다',
  '납치',
  '.',
  '감금',
  '만',
  '반복',
  '반복',
  '..',
  '이',
  '드라마',
  '는',
  '가족',
  '도',
  '없다',
  '연기',
  '못',
  '하는',
  '사람',
  '만

In [ ]:
# 토큰화 + 품사 부착
from konlpy.tag import Okt
okt = Okt()
oktTag = []

for text in df['document']:
  oktTag.append(okt.pos(text))

oktTag

[[('아', 'Exclamation'),
  ('더빙', 'Noun'),
  ('..', 'Punctuation'),
  ('진짜', 'Noun'),
  ('짜증나네요', 'Adjective'),
  ('목소리', 'Noun')],
 [('흠', 'Noun'),
  ('...', 'Punctuation'),
  ('포스터', 'Noun'),
  ('보고', 'Noun'),
  ('초딩', 'Noun'),
  ('영화', 'Noun'),
  ('줄', 'Noun'),
  ('....', 'Punctuation'),
  ('오버', 'Noun'),
  ('연기', 'Noun'),
  ('조차', 'Josa'),
  ('가볍지', 'Adjective'),
  ('않구나', 'Verb')],
 [('너', 'Modifier'),
  ('무재', 'Noun'),
  ('밓었', 'Noun'),
  ('다그', 'Noun'),
  ('래서', 'Noun'),
  ('보는것을', 'Verb'),
  ('추천', 'Noun'),
  ('한', 'Josa'),
  ('다', 'Adverb')],
 [('교도소', 'Noun'),
  ('이야기', 'Noun'),
  ('구먼', 'Noun'),
  ('..', 'Punctuation'),
  ('솔직히', 'Adjective'),
  ('재미', 'Noun'),
  ('는', 'Josa'),
  ('없다', 'Adjective'),
  ('..', 'Punctuation'),
  ('평점', 'Noun'),
  ('조정', 'Noun')],
 [('사이', 'Modifier'),
  ('몬페', 'Noun'),
  ('그', 'Determiner'),
  ('의', 'Noun'),
  ('익살스런', 'Adjective'),
  ('연기', 'Noun'),
  ('가', 'Josa'),
  ('돋보였던', 'Verb'),
  ('영화', 'Noun'),
  ('!', 'Punctuation'),
  ('스파이더맨', 'Nou

In [ ]:
from sklearn.base import validate_parameter_constraints
from collections import Counter

# 최빈 단어 추출
# most_word = []
# for text in df['document']:
#  most_word.append(Counter(text).most_common())

# most_word

# 불용어 제거
stopPos = ['Josa', 'Punctuation', 'Suffix', 'Alpha', 'Foreign', 'Unknown', 'KoreanParticle', 'ScreenName', 'Email', 'URL', 'Hashtag']
stopWord = ['로써', '다', '수', '등', '대', '작', '밓었', '무재']

word = []
for text in oktTag:
  # print(text)
  for tag in text:
    # print(tag)
    if tag[1] not in stopPos:
      if tag[0] not in stopWord and len(tag[0]) >= 2:
        # print(tag)
        word.append(tag[0])

In [ ]:
word

['더빙',
 '진짜',
 '짜증나네요',
 '목소리',
 '포스터',
 '보고',
 '초딩',
 '영화',
 '오버',
 '연기',
 '가볍지',
 '않구나',
 '다그',
 '래서',
 '보는것을',
 '추천',
 '교도소',
 '이야기',
 '구먼',
 '솔직히',
 '재미',
 '없다',
 '평점',
 '조정',
 '사이',
 '몬페',
 '익살스런',
 '연기',
 '돋보였던',
 '영화',
 '스파이더맨',
 '늙어',
 '보이기만',
 '했던',
 '커스틴',
 '던스트',
 '너무나도',
 '이뻐',
 '보였다',
 '걸음',
 '초등학교',
 '학년',
 '생인',
 '살용',
 '영화',
 '반개',
 '아까',
 '원작',
 '긴장감',
 '제대로',
 '살려내지못',
 '했다',
 '반개',
 '아깝다',
 '나온다',
 '이응경',
 '길용우',
 '생활',
 '정말',
 '해도',
 '그것',
 '낫겟다',
 '납치',
 '감금',
 '반복',
 '반복',
 '드라마',
 '가족',
 '없다',
 '연기',
 '하는',
 '사람',
 '액션',
 '없는데도',
 '재미',
 '있는',
 '안되는',
 '영화',
 '왜케',
 '평점',
 '낮은건데',
 '헐리우드',
 '화려함에만',
 '너무',
 '길들여져',
 '있나',
 '인피니트',
 '진짜',
 '볼때',
 '눈물나서',
 '죽겠다',
 '90년',
 '향수',
 '자극',
 '허진호',
 '감성',
 '절제',
 '멜로',
 '달인',
 '울면',
 '횡단보도',
 '건널',
 '쳐나',
 '이범수',
 '연기',
 '드럽게',
 '담백하고',
 '깔끔해서',
 '좋다',
 '로만',
 '보다',
 '보면',
 '자꾸',
 '잊어버린다',
 '사람',
 '이었다는',
 '취향',
 '존중',
 '다지',
 '진짜',
 '내생',
 '극장',
 '영화',
 '가장',
 '노잼',
 '감동',
 '스토리',
 '어거지',
 '감동',
 '어거지',
 '매번',
 '긴장',
 '

### LDA 모델링

In [ ]:
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

# 분석할 문서를 공백으로 토큰화
tokenized_docs = []
for doc in word:
  tokenized_docs.append(doc.split(' '))

tokenized_docs

[['더빙'],
 ['진짜'],
 ['짜증나네요'],
 ['목소리'],
 ['포스터'],
 ['보고'],
 ['초딩'],
 ['영화'],
 ['오버'],
 ['연기'],
 ['가볍지'],
 ['않구나'],
 ['다그'],
 ['래서'],
 ['보는것을'],
 ['추천'],
 ['교도소'],
 ['이야기'],
 ['구먼'],
 ['솔직히'],
 ['재미'],
 ['없다'],
 ['평점'],
 ['조정'],
 ['사이'],
 ['몬페'],
 ['익살스런'],
 ['연기'],
 ['돋보였던'],
 ['영화'],
 ['스파이더맨'],
 ['늙어'],
 ['보이기만'],
 ['했던'],
 ['커스틴'],
 ['던스트'],
 ['너무나도'],
 ['이뻐'],
 ['보였다'],
 ['걸음'],
 ['초등학교'],
 ['학년'],
 ['생인'],
 ['살용'],
 ['영화'],
 ['반개'],
 ['아까'],
 ['원작'],
 ['긴장감'],
 ['제대로'],
 ['살려내지못'],
 ['했다'],
 ['반개'],
 ['아깝다'],
 ['나온다'],
 ['이응경'],
 ['길용우'],
 ['생활'],
 ['정말'],
 ['해도'],
 ['그것'],
 ['낫겟다'],
 ['납치'],
 ['감금'],
 ['반복'],
 ['반복'],
 ['드라마'],
 ['가족'],
 ['없다'],
 ['연기'],
 ['하는'],
 ['사람'],
 ['액션'],
 ['없는데도'],
 ['재미'],
 ['있는'],
 ['안되는'],
 ['영화'],
 ['왜케'],
 ['평점'],
 ['낮은건데'],
 ['헐리우드'],
 ['화려함에만'],
 ['너무'],
 ['길들여져'],
 ['있나'],
 ['인피니트'],
 ['진짜'],
 ['볼때'],
 ['눈물나서'],
 ['죽겠다'],
 ['90년'],
 ['향수'],
 ['자극'],
 ['허진호'],
 ['감성'],
 ['절제'],
 ['멜로'],
 ['달인'],
 ['울면'],
 ['횡단보도'],
 ['건널'],
 ['쳐나'],
 ['이범수'],
 ['

In [ ]:
id2word = corpora.Dictionary(tokenized_docs)
print(id2word)

Dictionary<19832 unique tokens: ['더빙', '진짜', '짜증나네요', '목소리', '포스터']...>


In [ ]:
corpus_TDM = []
for doc in tokenized_docs:
  result = id2word.doc2bow(doc)
  corpus_TDM.append(result)

tokenized_docs, corpus_TDM

([['더빙'],
  ['진짜'],
  ['짜증나네요'],
  ['목소리'],
  ['포스터'],
  ['보고'],
  ['초딩'],
  ['영화'],
  ['오버'],
  ['연기'],
  ['가볍지'],
  ['않구나'],
  ['다그'],
  ['래서'],
  ['보는것을'],
  ['추천'],
  ['교도소'],
  ['이야기'],
  ['구먼'],
  ['솔직히'],
  ['재미'],
  ['없다'],
  ['평점'],
  ['조정'],
  ['사이'],
  ['몬페'],
  ['익살스런'],
  ['연기'],
  ['돋보였던'],
  ['영화'],
  ['스파이더맨'],
  ['늙어'],
  ['보이기만'],
  ['했던'],
  ['커스틴'],
  ['던스트'],
  ['너무나도'],
  ['이뻐'],
  ['보였다'],
  ['걸음'],
  ['초등학교'],
  ['학년'],
  ['생인'],
  ['살용'],
  ['영화'],
  ['반개'],
  ['아까'],
  ['원작'],
  ['긴장감'],
  ['제대로'],
  ['살려내지못'],
  ['했다'],
  ['반개'],
  ['아깝다'],
  ['나온다'],
  ['이응경'],
  ['길용우'],
  ['생활'],
  ['정말'],
  ['해도'],
  ['그것'],
  ['낫겟다'],
  ['납치'],
  ['감금'],
  ['반복'],
  ['반복'],
  ['드라마'],
  ['가족'],
  ['없다'],
  ['연기'],
  ['하는'],
  ['사람'],
  ['액션'],
  ['없는데도'],
  ['재미'],
  ['있는'],
  ['안되는'],
  ['영화'],
  ['왜케'],
  ['평점'],
  ['낮은건데'],
  ['헐리우드'],
  ['화려함에만'],
  ['너무'],
  ['길들여져'],
  ['있나'],
  ['인피니트'],
  ['진짜'],
  ['볼때'],
  ['눈물나서'],
  ['죽겠다'],
  ['90년'],
  ['향수'],
  ['자극'],
  [

In [ ]:
tfidf = TfidfModel(corpus_TDM) # Tfidf모델 선언
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽개수 - 변경 가능
lda = LdaModel(corpus=corpus_TFIDF,
               id2word=id2word,
               num_topics=n,
               random_state=100)

for t in lda.print_topics():
  print(t[0], ":", t[1])

0 : 0.082*"그냥" + 0.035*"있는" + 0.033*"역시" + 0.030*"가장" + 0.028*"재밌게" + 0.026*"주인공" + 0.025*"봤다" + 0.024*"보는" + 0.020*"인생" + 0.017*"낭비"
1 : 0.082*"진짜" + 0.071*"연기" + 0.058*"생각" + 0.033*"이건" + 0.031*"다시" + 0.030*"결말" + 0.028*"액션" + 0.026*"최악" + 0.025*"티비" + 0.020*"한번"
2 : 0.079*"드라마" + 0.054*"사람" + 0.049*"장면" + 0.046*"작품" + 0.026*"우리" + 0.020*"하지만" + 0.018*"가슴" + 0.017*"중간" + 0.017*"노잼" + 0.016*"추천"
3 : 0.060*"시간" + 0.060*"하나" + 0.049*"없는" + 0.043*"10" + 0.025*"없음" + 0.024*"보면" + 0.018*"나옴" + 0.016*"연기자" + 0.014*"기분" + 0.014*"싶은데"
4 : 0.076*"이런" + 0.070*"평점" + 0.052*"내용" + 0.046*"감독" + 0.039*"재미" + 0.024*"개봉" + 0.024*"마지막" + 0.023*"전혀" + 0.022*"한국" + 0.017*"보지"
5 : 0.172*"정말" + 0.058*"하는" + 0.052*"감동" + 0.022*"가족" + 0.019*"보는데" + 0.019*"마음" + 0.018*"이영화" + 0.014*"전개" + 0.014*"하면" + 0.013*"표현"
6 : 0.109*"너무" + 0.062*"보고" + 0.037*"배우" + 0.036*"없다" + 0.032*"쓰레기" + 0.025*"지루하고" + 0.023*"연출" + 0.023*"느낌" + 0.022*"봐도" + 0.021*"입니다"
7 : 0.046*"이렇게" + 0.035*"처음" + 0.034*"좋은" + 0.033*"같은" + 0.031*

### pyLDAvis로 시각화

In [ ]:
!pip install pyLDAvis==3.4.1

In [ ]:
# 그냥 하면 un-serialize 오류나서 피클링 진행
import pickle
pickle.dump(lda, open('lda.pkl', 'wb'))
pickle.load(open('lda.pkl', 'rb'))

In [ ]:
pickle.dump(corpus_TDM, open('corpus_TDM.pkl', 'wb'))
pickle.load(open('corpus_TDM.pkl', 'rb'))

[[(0, 1)],
 [(1, 1)],
 [(2, 1)],
 [(3, 1)],
 [(4, 1)],
 [(5, 1)],
 [(6, 1)],
 [(7, 1)],
 [(8, 1)],
 [(9, 1)],
 [(10, 1)],
 [(11, 1)],
 [(12, 1)],
 [(13, 1)],
 [(14, 1)],
 [(15, 1)],
 [(16, 1)],
 [(17, 1)],
 [(18, 1)],
 [(19, 1)],
 [(20, 1)],
 [(21, 1)],
 [(22, 1)],
 [(23, 1)],
 [(24, 1)],
 [(25, 1)],
 [(26, 1)],
 [(9, 1)],
 [(27, 1)],
 [(7, 1)],
 [(28, 1)],
 [(29, 1)],
 [(30, 1)],
 [(31, 1)],
 [(32, 1)],
 [(33, 1)],
 [(34, 1)],
 [(35, 1)],
 [(36, 1)],
 [(37, 1)],
 [(38, 1)],
 [(39, 1)],
 [(40, 1)],
 [(41, 1)],
 [(7, 1)],
 [(42, 1)],
 [(43, 1)],
 [(44, 1)],
 [(45, 1)],
 [(46, 1)],
 [(47, 1)],
 [(48, 1)],
 [(42, 1)],
 [(49, 1)],
 [(50, 1)],
 [(51, 1)],
 [(52, 1)],
 [(53, 1)],
 [(54, 1)],
 [(55, 1)],
 [(56, 1)],
 [(57, 1)],
 [(58, 1)],
 [(59, 1)],
 [(60, 1)],
 [(60, 1)],
 [(61, 1)],
 [(62, 1)],
 [(21, 1)],
 [(9, 1)],
 [(63, 1)],
 [(64, 1)],
 [(65, 1)],
 [(66, 1)],
 [(20, 1)],
 [(67, 1)],
 [(68, 1)],
 [(7, 1)],
 [(69, 1)],
 [(22, 1)],
 [(70, 1)],
 [(71, 1)],
 [(72, 1)],
 [(73, 1)],
 [(74, 

In [ ]:
pickle.dump(id2word, open('id2word.pkl', 'wb'))
pickle.load(open('id2word.pkl', 'rb'))

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus_TDM, id2word)
pyLDAvis.display(vis)

In [ ]:
# 토픽개수 3개로 바꿔서 한 번 더 시각화
n2 = 3
lda2 = LdaModel(corpus=corpus_TFIDF,
               id2word=id2word,
               num_topics=n2,
               random_state=100)

for t in lda2.print_topics():
  print(t[0], ":", t[1])

pyLDAvis.enable_notebook()
vis2 = pyLDAvis.gensim.prepare(lda2, corpus_TDM, id2word)
pyLDAvis.display(vis2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0 : 0.019*"너무" + 0.013*"이런" + 0.012*"평점" + 0.012*"스토리" + 0.012*"최고" + 0.011*"그냥" + 0.009*"내용" + 0.008*"하나" + 0.007*"재미" + 0.006*"이렇게"
1 : 0.101*"영화" + 0.026*"정말" + 0.014*"진짜" + 0.012*"연기" + 0.010*"생각" + 0.009*"하는" + 0.008*"감독" + 0.008*"감동" + 0.006*"없는" + 0.006*"이건"
2 : 0.011*"드라마" + 0.011*"보고" + 0.008*"시간" + 0.008*"사람" + 0.007*"장면" + 0.007*"배우" + 0.007*"작품" + 0.006*"없다" + 0.004*"사랑" + 0.004*"마지막"


## 실습 - 다른 풀이

**참고**
- iterrows : dataframe에서 idx, row 반환하는 함수
- map, apply, mapapply 활용해보기

### 토큰화 및 데이터 전처리

In [28]:
from konlpy.tag import Okt
tokenizer = Okt()

df['morphs'] = None
for i, row in df.iterrows():
  df['morphs'][i] = ' '.join(tokenizer.morphs(row['document']))

In [29]:
df.head()

,id,document,label,morphs
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 .. 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 ... 포스터 보고 초딩 영화 줄 .... 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 .. 솔직히 재미 는 없다 .. 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 ! 스파이더맨 에서 늙어 보이기만...


In [30]:
import re
# 한글을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-하-ㅣ\\s]")

def preprocess_remove(text):
  text = re.sub(remove_except_ko,' ',text).strip()
  return text

df['morphs'] = df['morphs'].map(lambda x : preprocess_remove(x))
df.head()

,id,document,label,morphs
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만...


In [31]:
# 불용어 처리
stop_word = ['포스터', '저작권자', '한경', '닷컴', '뉴스룸', '홈페이지', '바로', '네이버', '구독', '채널', '세요', '제공','께서', '라고', '입니다', '습니다', '한다', '에서', '이다', '에게', '으로', '이랑', '까지', '부터', '하다', '한데', '통해', '위해', '때문' ]

def preprocess(text):
  text = text.split()
  text = [i for i in text if len(i)>1]
  text = [i for i in text if i not in stop_word]
  return text

In [32]:
# 테스트
text = '물이 컵 모양을 하고 있을 때라도 결코 컵 모양에 구속되어 있지 않다. 컵 모양에 구속되어있다면 바가지에 들어가도 순식간에 바가지 모양으로 변하지 못한다. 컵 모양을 하고 있는 바로 그 때에도 컵 모양으로부터 자유로운 것이다. 우리의 본래 모습도 이와 같아서 그 어떤 처지나 상황에 있 더라도 그것에 구속되어 있지 않고 자유롭다. 비록 가난하더라도 나는 그것에 구속되어 있지 않고 자유롭다. 그래서 때가 되면 잘 살 수 있다. 성공과 실패, 칭찬과 비난, 그 어느 것에 대해서도 나는 구속되어 있지 않고 자유롭다. 나는 벼락에도 멍들지 않는 허공과 같다.'
preprocess(text)

['물이',
 '모양을',
 '하고',
 '있을',
 '때라도',
 '결코',
 '모양에',
 '구속되어',
 '있지',
 '않다.',
 '모양에',
 '구속되어있다면',
 '바가지에',
 '들어가도',
 '순식간에',
 '바가지',
 '모양으로',
 '변하지',
 '못한다.',
 '모양을',
 '하고',
 '있는',
 '때에도',
 '모양으로부터',
 '자유로운',
 '것이다.',
 '우리의',
 '본래',
 '모습도',
 '이와',
 '같아서',
 '어떤',
 '처지나',
 '상황에',
 '더라도',
 '그것에',
 '구속되어',
 '있지',
 '않고',
 '자유롭다.',
 '비록',
 '가난하더라도',
 '나는',
 '그것에',
 '구속되어',
 '있지',
 '않고',
 '자유롭다.',
 '그래서',
 '때가',
 '되면',
 '있다.',
 '성공과',
 '실패,',
 '칭찬과',
 '비난,',
 '어느',
 '것에',
 '대해서도',
 '나는',
 '구속되어',
 '있지',
 '않고',
 '자유롭다.',
 '나는',
 '벼락에도',
 '멍들지',
 '않는',
 '허공과',
 '같다.']

In [37]:
# 토큰화 + 토큰리스트 생성
def make_tokens(df):
  df['tokens'] = None
  for i, row in df.iterrows():
    if i%1000==0:
      print(i,'/',len(df))
    token = preprocess(df['morphs'][i])
    df['tokens'][i] = ' '.join(token)
  return df

df = make_tokens(df)

0 / 10000
1000 / 10000
2000 / 10000
3000 / 10000
4000 / 10000
5000 / 10000
6000 / 10000
7000 / 10000
8000 / 10000
9000 / 10000


In [34]:
df.head()

,id,document,label,morphs,tokens
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리,더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나,보고 초딩 영화 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다,무재 밓었 다그 래서 보는것을 추천
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정,교도소 이야기 구먼 솔직히 재미 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만...,사이 몬페 익살스런 연기 돋보였던 영화 스파이더맨 늙어 보이기만 했던 커스틴 던스트...


In [35]:
neg = (df[df['label']==0]).reset_index(drop=True)
pos = (df[df['label']==1]).reset_index(drop=True)
print('neg:', len(neg))
print('pos:', len(pos))

neg: 5021
pos: 4979


In [36]:
df.head()

,id,document,label,morphs,tokens
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리,더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나,보고 초딩 영화 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다,무재 밓었 다그 래서 보는것을 추천
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정,교도소 이야기 구먼 솔직히 재미 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만...,사이 몬페 익살스런 연기 돋보였던 영화 스파이더맨 늙어 보이기만 했던 커스틴 던스트...


### LDA 모델링

In [38]:
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

tokenized_docs = neg['tokens'].apply(lambda x : x.split())

id2word = corpora.Dictionary(tokenized_docs)
corpus_TDM = [id2word.doc2bow(doc) for doc in tokenized_docs]
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽개수 - 사용자가 직접 조절하기
lda = LdaModel(corpus=corpus_TFIDF,
               id2word=id2word,
               num_topics=n,
               random_state=100)

for t in lda.print_topics():
  print(t)

(0, '0.005*"영화" + 0.005*"뻔한" + 0.004*"ㅋㅋㅋㅋ" + 0.004*"졸작" + 0.004*"지루하다" + 0.004*"처음" + 0.003*"졸라" + 0.003*"별루" + 0.003*"ㅋㅋ" + 0.003*"ㅠㅠ"')
(1, '0.007*"영화" + 0.004*"감독" + 0.003*"드라마" + 0.003*"없고" + 0.003*"정말" + 0.003*"스토리" + 0.003*"액션" + 0.003*"진짜" + 0.003*"비디오" + 0.002*"ㅠㅠ"')
(2, '0.005*"진짜" + 0.004*"하나" + 0.004*"보다" + 0.004*"영화" + 0.003*"재미없어" + 0.003*"재미없음" + 0.003*"이래" + 0.003*"아까" + 0.003*"나옴" + 0.003*"누구"')
(3, '0.005*"너무" + 0.005*"영화" + 0.004*"코믹" + 0.003*"개인" + 0.003*"점점" + 0.003*"하하" + 0.003*"이건" + 0.003*"이란" + 0.003*"인데" + 0.003*"어색한"')
(4, '0.012*"영화" + 0.004*"코미디" + 0.004*"ㅡㅡ" + 0.003*"이딴" + 0.003*"이런" + 0.003*"공포영화" + 0.003*"최고" + 0.003*"이렇게" + 0.003*"취향" + 0.003*"시간"')
(5, '0.005*"영화" + 0.005*"노잼" + 0.004*"그냥" + 0.003*"스토리" + 0.003*"힘든" + 0.003*"개봉" + 0.003*"더럽게" + 0.003*"안된다" + 0.003*"그나마" + 0.003*"새끼"')
(6, '0.005*"재미" + 0.004*"영화" + 0.004*"제목" + 0.003*"없다" + 0.003*"없음" + 0.003*"긴장감" + 0.003*"도대체" + 0.003*"그닥" + 0.003*"삼류" + 0.003*"없고"')
(7, '0.011*"쓰레기" + 0.010*"영화" + 0

### 시각화

In [39]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus_TDM, id2word)
pyLDAvis.display(vis)

In [40]:
# 토픽개수 3개로 바꿔서 한 번 더 시각화
n2 = 3
lda2 = LdaModel(corpus=corpus_TFIDF,
               id2word=id2word,
               num_topics=n2,
               random_state=100)

for t in lda2.print_topics():
  print(t)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda2, corpus_TDM, id2word)
pyLDAvis.display(vis)

(0, '0.007*"영화" + 0.004*"없다" + 0.003*"진짜" + 0.003*"너무" + 0.003*"시간" + 0.002*"최악" + 0.002*"재미" + 0.002*"ㅋㅋ" + 0.002*"이런" + 0.002*"평점"')
(1, '0.006*"영화" + 0.005*"쓰레기" + 0.002*"아깝다" + 0.002*"ㅡㅡ" + 0.002*"내용" + 0.002*"스토리" + 0.002*"없는" + 0.002*"하고" + 0.002*"그냥" + 0.002*"같은"')
(2, '0.004*"별로" + 0.003*"영화" + 0.002*"정말" + 0.002*"보다" + 0.002*"이런" + 0.002*"스토리" + 0.002*"하나" + 0.002*"평점" + 0.002*"최악" + 0.001*"너무"')
